In [2]:
!pip install datasets  torch  transformers[torch] accelerate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 64.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.18.0+cu121 

In [3]:
from datasets import load_dataset, Dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, otherwise CPU
print(f"Using device: {device}")


dataset = load_dataset("Kaludi/Customer-Support-Responses")  # Loading   dataset

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/74 [00:00<?, ? examples/s]

In [4]:
from datasets import DatasetDict, Dataset, load_dataset, concatenate_datasets

train_size = int(len(dataset["train"]) * 0.95)  # 95% for training
eval_size = len(dataset["train"]) - train_size  # Remaining 5% for evaluation

train_dataset = dataset["train"].select(range(train_size))
eval_dataset = dataset["train"].select(range(train_size, train_size + eval_size))

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": eval_dataset,  # Using 'validation' as eval dataset key
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['query', 'response'],
        num_rows: 70
    })
    validation: Dataset({
        features: ['query', 'response'],
        num_rows: 4
    })
})


In [5]:
def preprocess_function(examples):
    # Tokenize the questions and answers
    inputs = tokenizer(examples["query"], padding="max_length", truncation=True, max_length=32)
    targets = tokenizer(examples["response"], padding="max_length", truncation=True, max_length=64)

    examples["input_ids"] = inputs.input_ids
    examples["attention_mask"] = inputs.attention_mask
    examples["labels"] = targets.input_ids  # Set the labels as the BART model's inputs

    return examples

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)  # Apply preprocessing function to the dataset

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['query', 'response', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 70
    })
    validation: Dataset({
        features: ['query', 'response', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
})

In [7]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=100,
    save_steps=500,
    eval_accumulation_steps=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    output_dir="./results",
)

# Initialize the BART model for sequence-to-sequence
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [8]:
# Defining the Seq2Seq trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)


trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,5.717162


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=18, training_loss=7.794409010145399, metrics={'train_runtime': 16.469, 'train_samples_per_second': 4.25, 'train_steps_per_second': 1.093, 'total_flos': 1333798502400.0, 'train_loss': 7.794409010145399, 'epoch': 1.0})

In [9]:
##INFERENCE

In [16]:
# Load the finetuned model for inference
model_path = "./results/checkpoint-18"  # Replace with your actual checkpoint
model = BartForConditionalGeneration.from_pretrained(model_path).to(device)

# Initialize the tokenizer again for safety (optional)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Example of inference
input_text = "I need to return an item"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate the answer
outputs = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Question:", input_text)
print("Answer:", decoded_output)


Question: I need to return an item
Answer: We'd be happy to help. Can you please provide your order number so we can send you the item you need?
